In [1]:
import psycopg2
import pickle
import qgrid

/home/lmoraes/miniconda/envs/machine_teaching/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [29]:
import pickle

with open("clustering_bcc.pkl", "rb") as pklfile:
    df = pickle.load(pklfile)

In [30]:
df

,X,coherence_k,coherence_med,coherence_samples,coherence_std,coherence_time,dataset,distance,gap,gap_std,...,is_binary,k,method,min_df,model,silhouette,silhouette_samples,silhouette_time,vectorizer,y
0,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",2,-1.025149,"[-1.33876579541, -0.711531716001]",0.313617,0.062945,bbc_5,cosine,-0.469492,0.002487,...,True,2,nmf,0.25,"NMF(alpha=0.0, beta_loss='frobenius', init=Non...",0.022251,"[-0.0244050748579, -0.175526370274, -0.1796602...",0.209427,CountVectorizer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [28]:
create_table_query = '''CREATE TABLE EXPERIMENTS
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    DATASET TEXT NOT NULL,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL,
    MIN_DF REAL NOT NULL,
    DISTANCE TEXT NOT NULL,
    K INTEGER NOT NULL,
    METHOD TEXT NOT NULL,
    MODEL TEXT NOT NULL,
    GAP REAL NOT NULL,
    GAP_STD REAL NOT NULL,
    GAP_TIME REAL NOT NULL,
    SILHOUETTE REAL NOT NULL,
    SILHOUETTE_SAMPLES REAL ARRAY NOT NULL,
    SILHOUETTE_TIME REAL NOT NULL,
    COHERENCE_SAMPLES REAL ARRAY NOT NULL,
    COHERENCE_MED REAL NOT NULL,
    COHERENCE_STD REAL NOT NULL,
    COHERENCE_TIME REAL NOT NULL,
    COHERENCE_K INTEGER NOT NULL)
    '''

create_table_query = '''CREATE TABLE EXPERIMENTS_TEMP
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL)
    '''

cursor.execute(create_table_query)
connection.commit()

In [33]:
query = cursor.mogrify(insert_query, query_values)
print(query)

b"INSERT INTO EXPERIMENTS_TEMP (X, y, vectorizer, is_binary) VALUES (ARRAY[ARRAY[0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0],ARRAY[0,0,0,0,0,1,1,1,0,1,1,0,1,0,0,0,0,1,0,1,0,1,0,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0],ARRAY[0,0,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,1,1,0,1,0,0],ARRAY[0,0,0,1,1,0,1,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,1,1,1,1,0,1,0,0,1,0,0,0,1,1,0,1,0],ARRAY[1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,1,1,1,1,0,0,1,1,1,0,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,1,0

In [34]:
cursor.execute(query)

In [9]:
import numpy as np

In [31]:
insert_query = "INSERT INTO EXPERIMENTS_TEMP "
column_value = []
insert_format = []
query_values = []
extra = []

for row in df.iterrows():
    query = []
#     for col in df.columns:
    for col in ["X", "y", "vectorizer", "is_binary"]:
        if isinstance(row[1][col], np.ndarray):
            query_values.append(row[1][col].tolist())
        else:
            query_values.append(row[1][col])
        column_value.append(col)
        insert_format.append("%s")
    break
    
insert_query += "(" + ", ".join(column_value) + ") VALUES "
insert_query += "(" + ", ".join(insert_format) + ")"
query_values = tuple(query_values)

In [32]:
insert_query

'INSERT INTO EXPERIMENTS_TEMP (X, y, vectorizer, is_binary) VALUES (%s, %s, %s, %s)'

In [25]:
query_values[1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [30]:
cursor.mogrify('INSERT INTO EXPERIMENTS_TEMP (X) VALUES (%s)', (d,))

ProgrammingError: can't adapt type 'numpy.ndarray'